In [10]:
from ska_sdp_func_python.calibration.dp3_calibration import create_parset_from_context, dp3_gaincal
from ska_sdp_func_python.sky_model.skymodel_imaging import skymodel_predict_calibrate

from ska_sdp_datamodels.sky_model.sky_functions import export_skymodel_to_text
from ska_sdp_datamodels.sky_model.sky_model import SkyComponent, SkyModel
from ska_sdp_datamodels.science_data_model.polarisation_model import PolarisationFrame
from ska_sdp_datamodels.configuration import create_named_configuration
from ska_sdp_datamodels.visibility import create_visibility

import numpy
from astropy.coordinates import SkyCoord
import astropy.units as u
import h5py
import matplotlib.pyplot as plt

In [11]:
def skycomponent():
    """Create a skycomponent to use for testing"""
    sky_pol_frame = "stokesIQUV"
    frequency = numpy.array([1.0e8])

    f = [100.0, 50.0, -10.0, 40.0]

    flux = numpy.outer(
        numpy.array([numpy.power(freq / 1e8, -0.7) for freq in frequency]),
        f,
    )

    compabsdirection = SkyCoord(
        ra=+181.0 * u.deg, dec=-35.0 * u.deg, frame="icrs", equinox="J2000"
    )
    comp = SkyComponent(
        direction=compabsdirection,
        frequency=frequency,
        flux=flux,
        polarisation_frame=PolarisationFrame(sky_pol_frame),
    )

    return comp


def create_visibility_single_source():
    """
    Visibility fixture
    """
    n_times = 2
    n_chan = 8
    low_core = create_named_configuration("LOW")
    times = (numpy.pi / 43200.0) * numpy.linspace(0.0, 300.0, n_times)
    frequency = numpy.linspace(1.0e8, 1.1e8, n_chan)
    channel_bandwidth = numpy.array([frequency[1] - frequency[0]] * n_chan)

    """Phase Centre fixture"""
    phase_centre = SkyCoord(
        ra=+180.0 * u.deg,
        dec=-35.0 * u.deg,
        frame="icrs",
        equinox="J2000",
    )
    
    vis = create_visibility(
        low_core,
        times,
        frequency,
        phasecentre=phase_centre,
        channel_bandwidth=channel_bandwidth,
        weight=1.0,
        polarisation_frame=PolarisationFrame("linear"),
    )
    return vis

In [12]:
visibility = create_visibility_single_source()
print(visibility.vis.data)

[[[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

  ...

  [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

  [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j 0

In [13]:
skymodel_vis = skymodel_predict_calibrate(visibility, SkyModel(components=skycomponent()))
print(skymodel_vis.vis.data)

[[[[ 1.50000000e+02  +0.j         -1.00000000e+01 +40.j
    -1.00000000e+01 -40.j          5.00000000e+01  +0.j        ]
   [ 1.50000000e+02  +0.j         -1.00000000e+01 +40.j
    -1.00000000e+01 -40.j          5.00000000e+01  +0.j        ]
   [ 1.50000000e+02  +0.j         -1.00000000e+01 +40.j
    -1.00000000e+01 -40.j          5.00000000e+01  +0.j        ]
   ...
   [ 1.50000000e+02  +0.j         -1.00000000e+01 +40.j
    -1.00000000e+01 -40.j          5.00000000e+01  +0.j        ]
   [ 1.50000000e+02  +0.j         -1.00000000e+01 +40.j
    -1.00000000e+01 -40.j          5.00000000e+01  +0.j        ]
   [ 1.50000000e+02  +0.j         -1.00000000e+01 +40.j
    -1.00000000e+01 -40.j          5.00000000e+01  +0.j        ]]

  [[ 1.10844998e+02-101.06130016j  1.95600135e+01 +36.29608618j
    -3.43393466e+01 -22.82124616j  3.69483327e+01 -33.68710005j]
   [ 1.18493125e+02 -91.97488447j  1.66270942e+01 +37.72982558j
    -3.24261775e+01 -25.46650765j  3.94977083e+01 -30.65829482j]
   [ 1.

In [14]:
export_skymodel_to_text(SkyModel(components=skycomponent()), "single_point.skymodel")
calibrated_vis = dp3_gaincal(skymodel_vis, ["B"], True, skymodel_filename = "single_point.skymodel", solutions_filename="uncorrupted.h5")

In [15]:
h5_solutions = h5py.File("uncorrupted.h5", "r")
amplitude_uncorrupted = h5_solutions["sol000/amplitude000/val"][:]

# print(amplitude_uncorrupted)
# plt.plot(amplitude_uncorrupted[:,0,0,0])
h5_solutions.close()

In [16]:
skymodel_vis.vis.data = 16 * skymodel_vis.vis.data
calibrated_corrupted_vis = dp3_gaincal(skymodel_vis, ["B"], True, skymodel_filename = "single_point.skymodel", solutions_filename="corrupted.h5")

In [17]:
h5_solutions = h5py.File("corrupted.h5", "r")
amplitude_corrupted = h5_solutions["sol000/amplitude000/val"][:]

# print(amplitude_corrupted)
# plt.plot(amplitude_corrupted[:,0,0,0])
h5_solutions.close()

In [18]:
amplitude_corrupted / amplitude_uncorrupted

array([[[[4.00001402, 3.99999393],
         [4.00001402, 3.99999393],
         [4.00001402, 3.99999393],
         ...,
         [4.00001402, 3.99999393],
         [4.00001402, 3.99999393],
         [4.00001402, 3.99999393]]],


       [[[4.00001402, 3.99999393],
         [4.00001402, 3.99999393],
         [4.00001402, 3.99999393],
         ...,
         [4.00001402, 3.99999393],
         [4.00001402, 3.99999393],
         [4.00001402, 3.99999393]]]])